In [1]:
import pandas as pd
import numpy as np

final = pd.read_excel("9.25final.xlsx")
df = pd.read_csv("parsed_training.csv")

/Users/zixuanzhang/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (194) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
columns = df.columns.tolist()
#Columns with missing value
columns_with_missing = []
for column in columns:
    if df[column].isnull().sum()>0:
        columns_with_missing.append({"column":column, "missing":df[column].isnull().sum()})
        
df_missing_distribution = pd.DataFrame.from_dict(columns_with_missing)
df_missing_distribution.sort_values("missing",ascending=False).reset_index(inplace=True)


from sklearn.impute import SimpleImputer
imputer_frequent = SimpleImputer(strategy="most_frequent")
for column in df_missing_distribution["column"]:
    try:
        if df[column].nunique()>=30:
            df[column].fillna(df[column].mean(),inplace=True)
        else:
            df[column].fillna(df[column].value_counts().index[0],inplace=True)
    except:
        print(column)
        

In [3]:
df1= df[final[final['Recursive_Feature_Selection2']==1]["index"]]
df1.shape

(69572, 816)

In [4]:
x = df1.drop(["transportation_issues"], axis=1)
x = pd.get_dummies(x)

y = df1["transportation_issues"]
y = y.astype("int")

In [5]:
from sklearn.model_selection import StratifiedShuffleSplit
 
split = StratifiedShuffleSplit(n_splits = 1,test_size = 0.2,random_state = 1)
for train_index,test_index in split.split(x,y):
    x_train = x.loc[train_index]
    y_train = y.loc[train_index]
    
    x_test = x.loc[test_index]
    y_test = y.loc[test_index]

In [6]:
# Upsampling
from sklearn.utils import resample

# from sklearn.utils import resample
X = x_train.copy()
X["transportation_issues"] = y_train
# separate minority and majority classes
neg_trans = X[X["transportation_issues"]==0]
pos_trans = X[X["transportation_issues"]==1]

# upsample minority
trans_sampled1 = resample(pos_trans,
                          replace=True, # sample with replacement
                          n_samples=len(neg_trans), # match number in majority class
                          random_state=40) # reproducible results

upsampled = pd.concat([neg_trans,trans_sampled1], axis = 0)

# check new class counts
upsampled["transportation_issues"].value_counts()

1    47499
0    47499
Name: transportation_issues, dtype: int64

In [7]:
# Split the data
x_train = pd.get_dummies(upsampled.iloc[:,:-1])
y_train = upsampled.iloc[:,-1]

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(interaction_only=True,include_bias = False)
x_train = poly.fit_transform(x_train)
x_test = poly.fit_transform(x_test)

In [ ]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(penalty="l2",solver="liblinear", max_iter=5000,
                        verbose=2, random_state=42).fit(x_train, y_train)
y_pred = clf.predict_proba(x_test)[:,1]

In [ ]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, y_pred)

In [ ]:
final
#final[final['Recursive_Feature_Addition']==1]["index"]